# Enter the dataset paths , the percentage for the data split and the parameters to train the model

### Before starting , once refer the readme.md file in the location "/home/viso/sample_dataset"

# If you want to download the predefined dataset follow the next 2 steps or else you can skip that

In [1]:
!python3 tools/misc/download_dataset.py --dataset-name cat --save-dir ./data/cat --unzip --delete

100%|████████████████████████████████████████| 217M/217M [00:46<00:00, 4.87MB/s]
Unzipping cat_dataset.zip
Delete data/cat/cat_dataset.zip


In [2]:
ls data/cat/annotations/

annotations_all.json  test.json  trainval.json


In [3]:
pwd

'/home/teric/Downloads/mmyolo'

In [5]:
#### dataset_root is the path , where images and the annotation file consists 
dataset_root=input('Enter the dataset root path')

# annotation_path , where there is a single file annotation in the coco (json) format 
annotation_path=input('Enter the annotation path ,(json file)')

# consists of images for 
images=input('Enter the path where images are there ,(excluding the dataset_root path )')

# The percentages are given to seperate the data into 3 parts (train , test and validation )
train_percentage=input('Enter the percentage to divide the train dataset . Eg:0.8')
test_percentage= input('Enter the percentage to divide the test dataset . Eg:0.1')
val_percentage= input('Enter the percentage to divide the validation dataset . Eg:0.1')

# enter the epochs , upto how much epochs the models needs to train 
MAX_EPOCHS = input('Enter the MAX_EPOCHS ')

# BATCH_SIZE means  "The number of training examples utilized in one iteration".
BATCH_SIZE = input('Enter the BATCH_SIZE')

#"Num classes" simply means the number of different categories in the dataset
# NUM_CLASSES = input('Enter the NUM_CLASSES ')

# select the model , need to do testing and quantization

In [6]:
import sys

# Dictionary of model types and their corresponding options
model_types = {
    "YOLOx": ["yolox_tiny", "yolox_s", "yolox_m", "yolox_l", "yolox_x"],
    "RTMDET": ["rtmdet_tiny", "rtmdet_s", "rtmdet_l", "rtmdet_m", "rtmdet_x"],
    "YOLOv7": ["yolov7_tiny", "yolov7_l", "yolov7_x", "yolov7_w", "yolov7_e"],
}

# Batch size options
batch_size_options = [1, 2, 4, 8, 16]

# Variable to store user's choice
selected_model_type = None
selected_model_variant = None
test_option = None
path = None
quantize_option = None
quantize_bits = None
batch_size = None
AUGMENTATIONS = {
    "noise": False,
    "denoise": False,
    "contrast": False,
    "brightness": False,
    "rotation": False
}
# Print model types
print("Please select a model type:")
for i, model_type in enumerate(model_types):
    print(f"{i}. {model_type}")

# Get input for model type
while True:
    model_type_choice = input("Enter choice (0-{}): ".format(len(model_types) - 1))
    try:
        model_type_choice = int(model_type_choice)
        if 0 <= model_type_choice < len(model_types):
            selected_model_type = list(model_types.keys())[model_type_choice]
            break
    except ValueError:
        pass

    print("Invalid choice, please try again")

# Print model variants for the selected model type
print(f"Please select a {selected_model_type} model variant:")
for i, model_variant in enumerate(model_types[selected_model_type]):
    print(f"{i}. {model_variant}")

# Get input for model variant
while True:
    model_variant_choice = input("Enter choice (0-{}): ".format(len(model_types[selected_model_type]) - 1))
    try:
        model_variant_choice = int(model_variant_choice)
        if 0 <= model_variant_choice < len(model_types[selected_model_type]):
            selected_model_variant = model_types[selected_model_type][model_variant_choice]
            break
    except ValueError:
        pass

    print("Invalid choice, please try again")

print(f"You selected: {selected_model_variant}")
# Ask if the user wants to train the model with data augmentation
while True:
    train_with_augmentation = input("Do you want to train the model with data augmentation? (0 for no, 1 for yes): ")
    if train_with_augmentation in ["0", "1"]:
        break
    else:
        print("Invalid choice, please enter '0' or '1'.")
if train_with_augmentation == "1":
    print("Please specify whether you want to enable or disable each augmentation option (0 for False, 1 for True):")
    for augmentation, enabled in AUGMENTATIONS.items():
        while True:
            choice = input(f"Do you want to enable {augmentation.capitalize()}? (0 for False, 1 for True): ")
            if choice in ["0", "1"]:
                AUGMENTATIONS[augmentation] = choice == "1"
                break
            else:
                print("Invalid choice, please enter '0' or '1'.")
# Ask if the user wants to test
while True:
    test_option = input("Do you need to test? (0 for no, 1 for yes): ")
    if test_option in ["0", "1"]:
        break
    else:
        print("Invalid choice, please enter '0' or '1'.")

# If testing is required, ask for image or video and corresponding path
if test_option == "1":
    while True:
        test_type = input("Do you want to test on an image or a video? (0 for image, 1 for video): ")
        if test_type == "0":
            path = ''  # You can set a default value or leave it empty for images
            break
        elif test_type == "1":
            video_path = input("Enter the video path: ")
            output_path = input("Enter the path to save the video: ")

            break
        else:
            print("Invalid choice, please enter '0' or '1'.")

# Now 'path' will be either an empty string for images or the user-provided video path for videos

# Ask if the user wants to quantize the model
while True:
    quantize_option = input("Do you want to quantize the model? (0 for no, 1 for yes): ")
    if quantize_option in ["0", "1"]:
        break
    else:
        print("Invalid choice, please enter '0' or '1'.")

# If quantizing is required, ask for the quantization bit precision
if quantize_option == "1":
    while True:
        quantize_bits = input("Choose quantization precision (0 for fp16, 1 for fp32): ")
        if quantize_bits in ["0", "1"]:
            if quantize_bits == "0" or quantize_bits == "1":
                while True:
                    print("Choose batch size:")
                    for i, size in enumerate(batch_size_options):
                        print(f"{i}. {size}")

                    batch_size_choice = input("Enter choice (0-{}): ".format(len(batch_size_options) - 1))
                    try:
                        batch_size_choice = int(batch_size_choice)
                        if 0 <= batch_size_choice < len(batch_size_options):
                            batch_size = batch_size_options[batch_size_choice]
                            break
                    except ValueError:
                        pass

                    print("Invalid choice, please try again.")

            break
        else:
            print("Invalid choice, please enter '0' or '1'.")

print(f"You selected: {selected_model_variant}")
print(f"Testing option: {'yes' if test_option == '1' else 'no'}")
if test_option == "1":
    print(f"Testing on: {'image' if test_type == '0' else 'video'} at path: {path}")
print(f"Quantize option: {'yes' if quantize_option == '1' else 'no'}")
if quantize_option == "1":
    print(f"Quantization precision: {'fp16' if quantize_bits == '0' else 'fp32'}")
    print(f"Batch size: {batch_size}")


Please select a model type:
0. YOLOx
1. RTMDET
2. YOLOv7
Please select a RTMDET model variant:
0. rtmdet_tiny
1. rtmdet_s
2. rtmdet_l
3. rtmdet_m
4. rtmdet_x
You selected: rtmdet_l
Please specify whether you want to enable or disable each augmentation option (0 for False, 1 for True):
Choose batch size:
0. 1
1. 2
2. 4
3. 8
4. 16
You selected: rtmdet_l
Testing option: yes
Testing on: video at path: None
Quantize option: yes
Quantization precision: fp32
Batch size: 4


In [7]:
import os
parent_directory = os.path.dirname(annotation_path)
parent_directory

'/home/teric/Downloads/mmyolo/data/cat/annotations'

In [8]:
AUGMENTATIONS

{'noise': False,
 'denoise': False,
 'contrast': False,
 'brightness': True,
 'rotation': True}

In [9]:
train_with_augmentation

'1'

In [10]:
dataset_root

'/home/teric/Downloads/mmyolo/data/cat'

# Generating metainfo for the config based on the annotation file 

In [11]:
import json

with open(annotation_path, 'r') as f:
    data = json.load(f)

classes = [c['name'] for c in data['categories']]
NUM_CLASSES = len(classes)
palette = []

for i, c in enumerate(classes):
    r = (i * 50) % 255
    g = (i * 100) % 255
    b = (i * 150) % 255
    palette.append((r, g, b))

metainfo = {
    'classes': classes,
    'palette': palette
}
class_names=metainfo['classes']
print("Class Names:", class_names)
print("Number of Classes:", NUM_CLASSES)
print("Metainfo:", metainfo)


Class Names: ['cat']
Number of Classes: 1
Metainfo: {'classes': ['cat'], 'palette': [(0, 0, 0)]}


In [12]:
NUM_CLASSES

1

# Data Augmentation 

# converting the annotations from json to txt  

In [13]:
import json
import shutil
# Load COCO annotations 
def convert_coco_to_yolo_old(annotation_path,dataset_root):
    print(".......... in convert_coco_to_yolo ..........")
    print(annotation_path)
    # Ensure the "txt_files" directory exists or create it
    txt_files_dir = os.path.join(dataset_root, 'txt_files')
    print(txt_files_dir)
    if not os.path.exists(txt_files_dir):
        os.makedirs(txt_files_dir)

    with open(annotation_path) as f:
        coco = json.load(f)


    # Create lists to store image info and annotations
    image_ids = []
    image_info = []
    annotations = []
    print("image_ids",image_ids)
    print("image_info",image_info)
    print("annotations",annotations)
    # Get image ids and image info 
    for img in coco['images']:
        image_ids.append(img['id'])
        image_info.append({'file_name': img['file_name'],
                           'width': img['width'],
                           'height': img['height']})
        # print("image id are",image_ids)
    # print(".........................")
    # Get annotations 
    for ann in coco['annotations']:
        bbox = ann['bbox']
        x, y, width, height = bbox
        image_id = ann['image_id']
        category_id = ann['category_id']
        # print("image id 'ssssssssss",image_id)
        # Convert to relative cooridnates 
        x_rel = x/image_info[image_id-1]['width'] 
        y_rel = y/image_info[image_id-1]['height']
        width_rel = width/image_info[image_id-1]['width']
        height_rel = height/image_info[image_id-1]['height']

        annotation = (category_id, x_rel, y_rel, width_rel, height_rel)
        annotations.append({'image_id': image_id, 'annotation': annotation})

    # Convert to YOLO format
    yolo_annotations = []
    for annotation in annotations:
        image_id = annotation['image_id']
        # print("image_id",image_id)
        category_id, x, y, width, height = annotation['annotation']

        # Calculate center x, y 
        x_center = x + width/2
        y_center = y + height/2

        yolo_annotation = f"{category_id} {x_center} {y_center} {width} {height}\n"
        yolo_annotations.append({'image_id': image_id, 'annotation': yolo_annotation})

    # Group annotations by image    
    yolo_files = {}
    for annotation in yolo_annotations:
        print(yolo_annotations)
        image_id = annotation['image_id']
        yolo_annotation = annotation['annotation']
        if image_id not in yolo_files:
            yolo_files[image_id] = [yolo_annotation]
        else:
            yolo_files[image_id].append(yolo_annotation)


    # Save annotation files
        for image_id, annotations in yolo_files.items(): 
            file_name = image_info[image_id-1]['file_name']
            # print('.1.',file_name)
            file_name = file_name.split('.')[0] + '.txt'
            # print('.2.',file_name)
            base_name = os.path.splitext(os.path.basename(file_name))[0]  # Get the base name without extension

            # Construct the destination .txt file name
            txt_file_name = base_name + '.txt'
            file_path = os.path.join(txt_files_dir, txt_file_name)

            #print(f"Saving to: {file_path}")

            with open(file_path, 'w') as f:
            # with open(txt_files_dir, 'w') as f:
                for annotation in annotations:
                    f.write(annotation)

            # with open(file_path, 'w') as f:
            #     for annotation in annotations:
            #         f.write(annotation + '\n')
            #         # destination_path = os.path.join(txt_files_dir, file_name)
            #         shutil.move(file_path, txt_files_dir)



    # Save image list file
    # with open('train.txt', 'w') as f:
    #     for image_id in image_ids:
    #         file_name = image_info[image_id-1]['file_name']
    #         f.write(file_name + '\n')
    
#convert_coco_to_yolo(annotation_path, dataset_root)


In [14]:
import json
import os
import shutil

def convert_coco_to_yolo(annotation_path, dataset_root):
    print(".......... in convert_coco_to_yolo ..........")
    print(annotation_path)

    # Ensure the "txt_files" directory exists or create it
    txt_files_dir = os.path.join(dataset_root, 'txt_files')
    print(txt_files_dir)
    if not os.path.exists(txt_files_dir):
        os.makedirs(txt_files_dir)

    with open(annotation_path) as f:
        coco = json.load(f)

    # Create lists to store image info and annotations
    image_info = {}
    annotations = []

    # Get image info
    for img in coco['images']:
        image_info[img['id']] = {'file_name': img['file_name'],
                                  'width': img['width'],
                                  'height': img['height']}

    # Get annotations 
    for ann in coco['annotations']:
        bbox = ann['bbox']
        x, y, width, height = bbox
        image_id = ann['image_id']
        if image_id not in image_info:
            print(f"Warning: Image ID {image_id} not found in image info")
            continue

        category_id = ann['category_id']
        
        # Convert to relative coordinates 
        x_rel = x / image_info[image_id]['width']
        y_rel = y / image_info[image_id]['height']
        width_rel = width / image_info[image_id]['width']
        height_rel = height / image_info[image_id]['height']

        annotation = (category_id, x_rel, y_rel, width_rel, height_rel)
        annotations.append({'image_id': image_id, 'annotation': annotation})

    # Convert to YOLO format
    yolo_annotations = []
    for annotation in annotations:
        image_id = annotation['image_id']
        category_id, x, y, width, height = annotation['annotation']

        # Calculate center x, y 
        x_center = x + width / 2
        y_center = y + height / 2

        yolo_annotation = f"{category_id} {x_center} {y_center} {width} {height}\n"
        yolo_annotations.append({'image_id': image_id, 'annotation': yolo_annotation})

    # Group annotations by image    
    yolo_files = {}
    for annotation in yolo_annotations:
        image_id = annotation['image_id']
        yolo_annotation = annotation['annotation']
        if image_id not in yolo_files:
            yolo_files[image_id] = [yolo_annotation]
        else:
            yolo_files[image_id].append(yolo_annotation)

    # Save annotation files
    for image_id, annotations in yolo_files.items(): 
        if image_id not in image_info:
            print(f"Error: Image ID {image_id} not found in image info")
            continue
            
        file_name = image_info[image_id]['file_name']
        base_name = os.path.splitext(os.path.basename(file_name))[0]  # Get the base name without extension
        txt_file_name = base_name + '.txt'
        file_path = os.path.join(txt_files_dir, txt_file_name)

        with open(file_path, 'w') as f:
            for annotation in annotations:
                f.write(annotation)

    # Save image list file
    # with open('train.txt', 'w') as f:
    #     for image_id in image_ids:
    #         file_name = image_info[image_id-1]['file_name']
    #         f.write(file_name + '\n')

# Now let's call the function
# annotation_path = "/home/viso/sample_data/annotations/total_annotations.json"
# dataset_root = "/home/viso/sample_data/augmented_images"
# convert_coco_to_yolo(annotation_path, dataset_root)


# creating the config file for the data augmentation 

In [15]:
def edit_and_save_config(images, dataset_root,AUGMENTATIONS):
    print("............edit_and_save_config..............")
    # %cd /home/viso/datasets/enhance_script
    import json
    new_config_path = '/root/workspace/mmyolo/quantize/dataset_enhance_config.json'
    original_config_path = '/root/workspace/mmyolo/quantize/dataset_enhance_config-dummy.json'
    # Load the original config
    with open(original_config_path) as f:
        config = json.load(f)

    # Modify the content as needed
    config["original_image_folder"] = f'{dataset_root}/{images}'
    config["original_label_folder"] = f'{dataset_root}/txt_files'
    config["enhance_image_folder"] = f'{dataset_root}/train'
    config["enhance_label_folder"] = f'{dataset_root}/train'
    config["augmentations"] = {
        "noise": AUGMENTATIONS['noise'],
        "denoise": AUGMENTATIONS['denoise'],
        "contrast": AUGMENTATIONS['contrast'],
        "brightness": AUGMENTATIONS['brightness'],
        "rotation": AUGMENTATIONS['rotation']
    }
    # Save the modified config to a new file
    with open(new_config_path, 'w') as f:
        json.dump(config, f, indent=2)

# Example usage:

# random_middle_part = 'images'
# dataset_root = '/home/viso/sample_dataset'  # Provide the actual path
# edit_and_save_config(original_config_path, new_config_path, random_middle_part, dataset_root)


# Getting all the image_paths in the txt file 

In [16]:
import os

def get_image_paths(folder_path, extensions=[".jpg", ".png"]):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in extensions):
                image_paths.append(os.path.join(root, file))
    return image_paths

def write_paths_to_file(image_paths, output_file=f"{dataset_root}/train.txt"):
    with open(output_file, "w") as file:
        for path in image_paths:
            file.write(path + "\n")

# if __name__ == "__main__":
# folder_path = "/home/viso/sample_data/augmented_imagges"  # Replace with the actual path to your folder
    # image_paths = get_image_paths(folder_path)
    # write_paths_to_file(image_paths)


In [17]:
import os

def update_config_with_class_names(class_names):
    dummy_main = '/root/workspace/mmyolo/quantize/dummy_main.py'
    main='/root/workspace/mmyolo/quantize/main.py'
    with open(dummy_main, 'r') as f:
        cfg_content = f.read()

    # Replace specific lines with new values
    cfg_content = cfg_content.replace("classes =", f"classes = {class_names}")

    # Write the updated content back to the configuration file
    with open(main, 'w') as custom_cfg_file:
        custom_cfg_file.write(cfg_content)

# Example usage:
# number_of_classes = 10  # Replace with the actual number of classes


In [ ]:
import shutil
if train_with_augmentation=='1':
    # %cd /home/viso/notebooks
    # %run ./aug.ipynb
    convert_coco_to_yolo(annotation_path, dataset_root)
    edit_and_save_config(images, dataset_root,AUGMENTATIONS)
    %cd /root/workspace/mmyolo/quantize
    !python run_dataset_enhance.py
    # %cd /home/viso/github/Yolo-to-COCO-format-converter
    update_config_with_class_names(class_names)
    !mkdir output
    !pip install imagesize
    !python main.py --path {dataset_root} --output output_json.json
    %pwd
    shutil.copy('/root/workspace/mmyolo/quantize/output/output_json.json',parent_directory)
    %cd {dataset_root}
    folder_path=f'{dataset_root}/train'
    image_paths = get_image_paths(folder_path)
    write_paths_to_file(image_paths)
    with open('object.names', 'w') as f:
        for class_name in class_names:
            f.write(class_name + '\n')

In [ ]:
if train_with_augmentation=='1':
    annotation_path = f'{parent_directory}/output_json.json'
    print(annotation_path)
    images = 'train'

In [ ]:
print(type(AUGMENTATIONS))

In [ ]:
AUGMENTATIONS['noise']

# split the annotation file into train , test and validation 

### If there is single annotation file , that can divide into train , test and validation (coco format)

In [ ]:
pwd

In [ ]:
#!pip install sklearn
!pip install funcy
!pip install argparse
!pip install scikit-learn

In [ ]:
%cd /root/workspace/mmyolo

In [ ]:
!python cocosplit_train_test_valid.py \
    --annotations "{annotation_path}" \
    --train_ratio {train_percentage} \
    --valid_ratio {val_percentage} \
    --test_ratio {test_percentage} \
    --trainJson_name train.json \
    --validJson_name val.json \
    --testJson_name test.json


In [ ]:
pwd

In [ ]:
import os , shutil
# dataset_root = '/home/viso/datasets/viso_datasets/construction-demo'

# parent_directory = os.path.dirname(annotation_path)
shutil.copy('/root/workspace/mmyolo/test.json',parent_directory)
shutil.copy('/root/workspace/mmyolo/val.json',parent_directory)
shutil.copy('/root/workspace/mmyolo/train.json',parent_directory)

train_json = os.path.join(parent_directory, 'train.json')
val_json = os.path.join(parent_directory, 'val.json')
test_json = os.path.join(parent_directory, 'test.json')

# Remove the dataset_root prefix using os.path.relpath
train_json_relpath = os.path.relpath(train_json, dataset_root)
val_json_relpath = os.path.relpath(val_json, dataset_root)
test_json_relpath = os.path.relpath(test_json, dataset_root)

print(train_json_relpath, val_json_relpath, test_json_relpath)
print(parent_directory)

# Yolox configuration setup 

In [ ]:
# Check if the selected model type is 'YOLOx'
if selected_model_type == 'YOLOx':
    # Provide the path to the user for modification
    dummy_config = '/root/workspace/mmyolo/configs/yolox/custom_config.py'
    custom_config_file_path_base = '/root/workspace/mmyolo/configs/yolox/yolox_s_fast_8xb8-300e_coco.py'  # Set your desired path

    # Load the YOLOx config file
    with open(dummy_config, 'r') as f:
        cfg_content = f.read()

    # Replace specific lines with new values
    cfg_content = cfg_content.replace("data_root =", f"data_root = '{dataset_root}/'")
    cfg_content = cfg_content.replace("train_data_prefix =", f"train_data_prefix = '{images}'")
    cfg_content = cfg_content.replace("val_data_prefix =", f"val_data_prefix = '{images}'")
    cfg_content = cfg_content.replace("train_ann_file =", f"train_ann_file = '{train_json_relpath}'")
    cfg_content = cfg_content.replace("val_ann_file =", f"val_ann_file = '{val_json_relpath}'")
    cfg_content = cfg_content.replace("num_classes =", f"num_classes = {NUM_CLASSES}")
    cfg_content = cfg_content.replace("meta_info =", f"meta_info = {metainfo}")
    cfg_content = cfg_content.replace("max_epochs =", f"max_epochs = {MAX_EPOCHS}")
    cfg_content = cfg_content.replace("train_batch_size_per_gpu =", f"train_batch_size_per_gpu = {BATCH_SIZE}")

    # Save the modified content to the new file
    with open(custom_config_file_path_base, 'w') as custom_cfg_file:
        custom_cfg_file.write(cfg_content)

    print(f"Custom config file has been created at {custom_config_file_path_base}.")
    print("dummy_config:", dummy_config)
    print("custom_config_file_path_base:", custom_config_file_path_base)


# Yolov7 configuration setup 

In [ ]:
if selected_model_type == 'YOLOv7':
    dummy_config = '/root/workspace/mmyolo/configs/yolov7/custom_config.py'
    custom_config_file_path_base = '/root/workspace/mmyolo/configs/yolov7/yolov7_l_syncbn_fast_8x16b-300e_coco.py'  # Set your desired path

    # Load the YOLOx config file
    with open(dummy_config, 'r') as f:
        cfg_content = f.read()

    # Replace specific lines with new values
    cfg_content = cfg_content.replace("data_root =", f"data_root = '{dataset_root}/'")
    cfg_content = cfg_content.replace("train_data_prefix =", f"train_data_prefix = '{images}'")
    cfg_content = cfg_content.replace("val_data_prefix =", f"val_data_prefix = '{images}'")
    cfg_content = cfg_content.replace("train_ann_file =", f"train_ann_file = '{train_json_relpath}'")
    cfg_content = cfg_content.replace("val_ann_file =", f"val_ann_file = '{val_json_relpath}'")
    cfg_content = cfg_content.replace("num_classes =", f"num_classes = {NUM_CLASSES}")
    cfg_content = cfg_content.replace("meta_info =", f"meta_info = {metainfo}")
    cfg_content = cfg_content.replace("max_epochs =", f"max_epochs = {MAX_EPOCHS}")
    cfg_content = cfg_content.replace("train_batch_size_per_gpu =", f"train_batch_size_per_gpu = {BATCH_SIZE}")

    # Save the modified content to the new file
    with open(custom_config_file_path_base, 'w') as custom_cfg_file:
        custom_cfg_file.write(cfg_content)

    print(f"Custom config file has been created at {custom_config_file_path_base}.")

# Rtmdet configuration setup

In [ ]:
if selected_model_type == 'RTMDET':
    dummy_config = '/root/workspace/mmyolo/configs/rtmdet/custom_config.py'
    custom_config_file_path_base = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_l_syncbn_fast_8xb32-300e_coco.py'  # Set your desired path

    # Load the YOLOx config file
    with open(dummy_config, 'r') as f:
        cfg_content = f.read()

    # Replace specific lines with new values
    cfg_content = cfg_content.replace("data_root =", f"data_root = '{dataset_root}/'")
    cfg_content = cfg_content.replace("train_data_prefix =", f"train_data_prefix = '{images}'")
    cfg_content = cfg_content.replace("val_data_prefix =", f"val_data_prefix = '{images}'")
    cfg_content = cfg_content.replace("train_ann_file =", f"train_ann_file = '{train_json_relpath}'")
    cfg_content = cfg_content.replace("val_ann_file =", f"val_ann_file = '{val_json_relpath}'")
    cfg_content = cfg_content.replace("num_classes =", f"num_classes = {NUM_CLASSES}")
    cfg_content = cfg_content.replace("meta_info =", f"meta_info = {metainfo}")
    cfg_content = cfg_content.replace("max_epochs =", f"max_epochs = {MAX_EPOCHS}")
    cfg_content = cfg_content.replace("train_batch_size_per_gpu =", f"train_batch_size_per_gpu = {BATCH_SIZE}")

    # Save the modified content to the new file
    with open(custom_config_file_path_base, 'w') as custom_cfg_file:
        custom_cfg_file.write(cfg_content)

    print(f"Custom config file has been created at {custom_config_file_path_base}.")
    print("dummy_config:", dummy_config)


In [ ]:
if selected_model_variant=='yolox_tiny':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_tiny_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolox_s':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_s_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolox_m':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_m_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolox_l':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_l_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolox_x':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_x_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolov7_tiny':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_tiny_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'yolov7_l':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_l_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'yolov7_x':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_x_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'yolov7_w':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_w-p6_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'yolov7_e':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_e_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'rtmdet_tiny':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_tiny_syncbn_fast_8xb32-300e_coco.py'
elif selected_model_variant == 'rtmdet_s':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_s_syncbn_fast_8xb32-300e_coco.py'
elif selected_model_variant == 'rtmdet_l':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_l_syncbn_fast_8xb32-300e_coco.py'
elif selected_model_variant == 'rtmdet_m':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_m_syncbn_fast_8xb32-300e_coco.py'
elif selected_model_variant == 'rtmdet_x':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_x_syncbn_fast_8xb32-300e_coco.py'

In [ ]:
custom_config_file_path=config_file_path.split('/')[-1]

# train the data with the selected model 

In [ ]:
%cd /root/workspace/mmyolo/

In [ ]:
selected_model_type=selected_model_type.lower()
print(selected_model_type)
custom_config_file_path

In [ ]:
MAX_EPOCHS

In [ ]:
import subprocess

# Assuming config_file_path is something like "yolox/config_file.py"
# config_file_path = config_file_path.split('/')[-1]
command_line = f"python tools/train.py configs/{selected_model_type}/{custom_config_file_path}"

# Run the command
result = subprocess.run(command_line, shell=True)

# Check the result
if result.returncode == 0:
    print("Command executed successfully")
else:
    print(f"Error executing the command. Return code: {result.returncode}")
    print(f"Output: {result.stdout}")
    print(f"Error: {result.stderr}")


# testing with the trained model with image

In [ ]:
import os
# Remove the last .py extension
custom_config_file_path_without_extension = os.path.splitext(custom_config_file_path)[0]

print(custom_config_file_path_without_extension)


In [ ]:
test_images=images

In [ ]:
metainfo

In [ ]:
if test_option == '1' and test_type == '0':
    %cd /root/workspace/mmyolo
    %run ./test.ipynb
    testing_image(MAX_EPOCHS, custom_config_file_path_base,test_images,test_json_relpath,dummy_config,metainfo,BATCH_SIZE,NUM_CLASSES,dataset_root)   
    %cd /root/workspace/mmyolo
    import subprocess
    command_line = f"python tools/test.py \
        configs/{selected_model_type}/{custom_config_file_path} \
        work_dirs/{custom_config_file_path_without_extension}/epoch_{MAX_EPOCHS}.pth \
        --work-dir wk"

    # Run the command
    result = subprocess.run(command_line, shell=True)

    # Check the result
    if result.returncode == 0:
        print("Command executed successfully")
    else:
        print(f"Error executing the command. Return code: {result.returncode}")
        print(f"Output: {result.stdout}")
        print(f"Error: {result.stderr}")

# testing with the trained model with video

In [ ]:
if test_option == '1' and test_type == '1':
    print(".....testing on video......")
    %cd /root/workspace/mmyolo
    %run ./test.ipynb
    testing_video(MAX_EPOCHS, custom_config_file_path_base,selected_model_type,video_path,output_path,custom_config_file_path)

# Quantize the model (convert the pytorch model into tensorrt model )

In [ ]:
if quantize_option == '1':
    %cd /root/workspace/mmyolo
    %run ./quantize.ipynb
    quantize(quantize_option, quantize_bits, batch_size_choice)
    import subprocess
    %cd /root/workspace/
    command_line = f"python mmdeploy/tools/deploy.py \
    /root/workspace/mmyolo/configs/deploy/detection_tensorrt_dynamic-192x192-960x960.py \
    {config_file_path} \
    mmyolo/work_dirs/{custom_config_file_path_without_extension}/epoch_{MAX_EPOCHS}.pth \
    mmyolo/demo/demo.jpg \
    --work-dir mmdeploy_model/{custom_config_file_path}_tensorrt \
    --device cuda \
    --dump-info"

    # Run the command
    result = subprocess.run(command_line, shell=True)

    # Check the result
    if result.returncode == 0:
        print("Command executed successfully")
    else:
        print(f"Error executing the command. Return code: {result.returncode}")
        print(f"Output: {result.stdout}")
        print(f"Error: {result.stderr}")